这篇博客（[视频](https://www.bilibili.com/video/BV1co4y1p7Sf/?spm_id_from=333.999.0.0&vd_source=b4496ba547455f3db613a8f8de6bea01)）记录从巨潮资讯爬取相关公告的 PDF 文件。
<!-- TEASER_END -->

In [1]:
import logging
import os
import time
import requests
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

In [2]:
data = pd.read_csv('./data/final.csv', dtype={'secCode':str})

In [3]:
data.head()

,secCode,secName,orgId,announcementTitle,announcementTime,adjunctUrl,announcementContent
0,002152,广电运通,9900003423,广电运通：2016年4月1日投资者关系活动记录表,2016-04-06,http://static.cninfo.com.cn/finalpage/2016-04-...,9、公司提过有储备区块链技术，是否可以介绍？
1,002177,御银股份,9900003781,御银股份：2016年4月21日投资者关系活动记录表,2016-04-21,http://static.cninfo.com.cn/finalpage/2016-04-...,4、公司在区块链技术上未来有什么规划？
2,300386,飞天诚信,9900023058,飞天诚信：2016年4月22日投资者关系活动记录表,2016-04-25,http://static.cninfo.com.cn/finalpage/2016-04-...,公司在区块链部分如何布局?
3,300423,鲁亿通,9900028812,鲁亿通：关于本次交易前12个月内购买、出售资产的说明,2016-06-09,http://static.cninfo.com.cn/finalpage/2016-06-...,同时，上市公司拟采用定价发行的方式向纪法清、孔剑平、孙奇锋、王麒诚和中信建投-数贝泽华人工智...
4,000961,中南建设,gssz0000961,中南建设：关于本公司投资上海承泰信息科技股份有限公司的进展公告,2016-06-23,http://static.cninfo.com.cn/finalpage/2016-06-...,，以区块链技术处理玛娜花园不同版本的个人数据，形成对数据信息的安全保护。


In [4]:
data.iloc[4,5]

'http://static.cninfo.com.cn/finalpage/2016-06-23/1202387515.PDF'

In [5]:
def get_announcement(s, url, secName):
    try:
        r = s.get(url)
        filename = str(secName) + ".pdf" 
         # 注意要用'wb', b表示二进制，不要用'w'
        with open(filename, 'wb') as f: 
             # r.content -> requests中的二进制响应内容：以字节的方式访问请求响应体
            f.write(r.content)                                 
    except Exception as e:
        logging.warning(secName, e)
        time.sleep(random.random()*5)

In [6]:
os.chdir("announcements")
url = data.loc[:, "adjunctUrl"]
secName = data.loc[:, "secName"]

with ThreadPoolExecutor(max_workers=8) as pool:
    with requests.Session() as s:
        futures = [pool.submit(get_announcement, s, url, secName) for url,secName in zip(url, secName)]